In [1]:
import numpy as np
import unicodedata
import inspect

# Levenshtein edit distance

In [2]:
from edit_distance import levenshtein_matrix, levenshtein

print(inspect.getsource(levenshtein_matrix))
print(inspect.getsource(levenshtein))

def levenshtein_matrix(seq1, seq2):
    """Compute the matrix commonly computed to produce the Levenshtein distance.

    This is also known as the Wagner-Fischer algorithm. The matrix element at the bottom right contains the desired
    edit distance.

    This algorithm is implemented here because we need an implementation that can work with sequences other than
    strings, e.g. lists of grapheme clusters or lists of word strings.
    """
    m = len(seq1)
    n = len(seq2)

    def from_to(start, stop):
        return range(start, stop + 1, 1)

    D = np.zeros((m + 1, n + 1), np.int)
    D[0, 0] = 0
    for i in from_to(1, m):
        D[i, 0] = i
    for j in from_to(1, n):
        D[0, j] = j
    for i in from_to(1, m):
        for j in from_to(1, n):
            D[i, j] = min(
                D[i - 1, j - 1] + 1 * (seq1[i - 1] != seq2[j - 1]),  # Same or Substitution
                D[i, j - 1] + 1,  # Insertion
                D[i - 1, j] + 1   # Deletion
            )

    ret

In [3]:
assert levenshtein('a', 'a') == 0
assert levenshtein('a', 'b') == 1
assert levenshtein('Foo', 'Bar') == 3
assert levenshtein('', '') == 0
assert levenshtein('Foo', '') == 3
assert levenshtein('', 'Foo') == 3
assert levenshtein('Fnord', 'Food') == 2
assert levenshtein('Müll', 'Mull') == 1
assert levenshtein('Abstand', 'Sand') == 4

This fails for different representations of the "same" canonically equivalent string:

In [4]:
word1 = unicodedata.normalize('NFC', 'Schlyñ')
word2 = unicodedata.normalize('NFD', 'Schlyñ')  # Different, decomposed!
levenshtein(word1, word2)

2

In [5]:
# Same, but for grapheme clusters
from uniseg.graphemecluster import grapheme_clusters

word1 = list(grapheme_clusters(unicodedata.normalize('NFC', 'Schlyñ')))
word2 = list(grapheme_clusters(unicodedata.normalize('NFD', 'Schlyñ')))
levenshtein(word1, word2)

1

Better.

Let's define a edit distance function that uses the basic Levenshtein algorithm, but knows about Unicode normalization and grapheme clusters!

In [6]:
from edit_distance import distance
print(inspect.getsource(distance))

def distance(s1, s2):
    """Compute the Levenshtein edit distance between two Unicode strings

    Note that this is different from levenshtein() as this function knows about Unicode normalization and grapheme
    clusters. This should be the correct way to compare two Unicode strings.
    """
    s1 = list(grapheme_clusters(unicodedata.normalize('NFC', s1)))
    s2 = list(grapheme_clusters(unicodedata.normalize('NFC', s2)))
    return levenshtein(s1, s2)



In [7]:
word1 = unicodedata.normalize('NFC', 'Schlyñ')
word2 = unicodedata.normalize('NFD', 'Schlyñ')  # Different, decomposed!

distance(word1, word2)

0

This should give us the correct answer of 1 for 'Schlyñ' (with LATIN SMALL LETTER N WITH TILDE) vs 'Schlym̃' (with LATIN SMALL LETTER M + COMBINING TILDE):

In [8]:
word1 = 'Schlyñ'
word2 = 'Schlym̃'
#print('Lengths, as far as Python is concerned:', len(word1), len(word2))  # → gives 6 and 7!
distance(word1, word2)

1

# Edit operations

python-Levenshtein supports backtracing, i.e. giving a sequence of edit options that transforms a word to another word:



In [9]:
import Levenshtein
word1 = 'Schlyñ'  # with LATIN SMALL LETTER N WITH TILDE
word2 = 'Schlym̃'  # with LATIN SMALL LETTER M + COMBINING TILDE
print(Levenshtein.editops(word1, word2))

[('insert', 5, 5), ('replace', 5, 6)]


Note that it does not work with grapheme clusters, but "characters", so it gives 2 operations.

Defining our own `editops()`. (This looks a bit wild due to our own tail recursion handling.)

In [10]:
from edit_distance import seq_editops, editops
print(inspect.getsource(seq_editops))
print(inspect.getsource(editops))

def seq_editops(seq1, seq2):
    seq1 = list(seq1)
    seq2 = list(seq2)
    m = len(seq1)
    n = len(seq2)
    D = levenshtein_matrix(seq1, seq2)

    def _tail_backtrace(i, j, accumulator):
        if i > 0 and D[i - 1, j] + 1 == D[i, j]:
            return partial(_tail_backtrace, i - 1, j, [('delete', i-1, j)] + accumulator)
        if j > 0 and D[i, j - 1] + 1 == D[i, j]:
            return partial(_tail_backtrace, i, j - 1, [('insert', i, j-1)] + accumulator)
        if i > 0 and j > 0 and D[i - 1, j - 1] + 1 == D[i, j]:
            return partial(_tail_backtrace, i - 1, j - 1, [('replace', i-1, j-1)] + accumulator)
        if i > 0 and j > 0 and D[i - 1, j - 1] == D[i, j]:
            return partial(_tail_backtrace, i - 1, j - 1, accumulator)  # NOP
        return accumulator

    def backtrace(i, j):
        result = partial(_tail_backtrace, i, j, [])
        while isinstance(result, partial):
            result = result()

        return result

    b = backtrace(m, n)
    re

In [11]:
editops('Foo', 'Fon')

[('replace', 2, 2)]

In [12]:
print(editops('Käptn', 'Käpt\'n'))
print(Levenshtein.editops('Käptn', 'Käpt\'n'))

[('insert', 4, 4)]
[('insert', 4, 4)]


In [13]:
print(editops('Delete something', 'Deletesomething'))
print(Levenshtein.editops('Delete something', 'Deletesomething'))

[('delete', 6, 6)]
[('delete', 6, 6)]


In [14]:
print(editops('A more difficult example', 'Amore difficült  exampl'))
print(Levenshtein.editops('A more difficult example', 'Amore difficült  exampl'))

[('delete', 1, 1), ('replace', 13, 12), ('insert', 17, 16), ('delete', 23, 23)]
[('delete', 1, 1), ('replace', 13, 12), ('insert', 16, 15), ('delete', 23, 23)]


XXX Note that our implementation returns different positions here for the 'insert'. 

Let's try it with a difficult example that needs grapheme cluster handling:

In [15]:
word1 = 'Schlyñ'  # with LATIN SMALL LETTER N WITH TILDE
word2 = 'Schlym̃'  # with LATIN SMALL LETTER M + COMBINING TILDE

editops(word1, word2)

[('replace', 5, 5)]

🎉

# Character error rate

[digitisation.eu](https://sites.google.com/site/textdigitisation/qualitymeasures/computingerrorrates) defines the character error rate (CER) as:

$$
\text{CER} = \frac{i + s + d}{n}
$$

where $i$ is the number of inserts, $s$ the number of substitutions, $d$ the number of deletions and $n$ is the number of characters in the reference text. (The text is not super clear about $n$ being the number of characters in the reference text, but it seems appropiate as they *are* clear about this when computing the word error rate.)

Because our edit distance is equal to $i + s + d$, we can thus define:

In [16]:
from character_error_rate import character_error_rate
print(inspect.getsource(character_error_rate))

def character_error_rate(reference, compared):
    d = distance(reference, compared)
    if d == 0:
        return 0

    n = len(list(grapheme_clusters(unicodedata.normalize('NFC', reference))))
    if n == 0:
        return float('inf')

    return d/n



In [17]:
assert character_error_rate('Foo', 'Bär') == 3/3
assert character_error_rate('Fnord', 'Food') == 2/5
assert character_error_rate('Food', 'Fnord') == 2/4
assert character_error_rate('Schlyñ', 'Schlym̃') == 1/6

In [18]:
# From experiments/2019-07-ocrevalUAtion: These are already preprocessed by the equivalences in equivalences-tess-frk.csv.
gt = """115 über die vielen Sorgen wegen deſſelben vergaß Hartkopf, der Frau Amtmännin das ver⸗ ſprochene zu überliefern. — Ein Erpreſſer wurde an ihn abgeſchickt, um ihn ums Him⸗ melswillen zu ſagen, daß er das Verſprochene gleich den Augenblick überbringen möchte, die Frau Amtmännin hätte ſich auf ihn verlaſſen, und nun wüßte ſie nicht, was ſie anfangen ſollte. Den Augenblick ſollte er kommen, ſonſt vergieng ſie in ihrer Angſt. — Die Gäſte wären ſchon angekommen, und es fehlte ihr doch noch an allem. — Hartkopf mußte ſich erſt beſinnen, und endlich nach langem Nachdenken fiel es ihm erſt wieder ein. — Er langte den Zettel aus dem Accisbuche heraus, und ſagte ſeiner Frau, daß ſie das, was da wäre, herbeyſchaffen möchte. Jndeß mangelten doch einige Generalia, die alſo wegfielen. — Hartkopf gieng ſelbſt mit und überbrachte es. — „Herr Jemine! er böſer Mann!“ — ſchrie ihm die Frau Amtmännin entgegen, und ſchlug ihn auf die Schulter und blickte den Korb, der voll gedrückt, gerüttelt und überﬂüſſig in ihren Schoos gegeben werden ſollte, mit Augen voller Freu⸗ H 2"""
tess = """emm unmit; Lis Übey die vielen Sorgen wegen" deſſelben vergaß Hartkopf, der Frau! Amimännin das- ver ſprochene zu überliefeen. ==" Ein Epypreſſer- wurde an ihn abgeſchieet', um' ihn ums Hime melswillen zu ſagen, "daß er das Verſyrochene leich den Augenblick "überbringen möchte, die Frau Amtmännin hätte ſich auf ihn veriaſſen, und nun wüßte ſie- nicht, was ſie anfangen ſollte, =! 'Den Augenblick ſollte "er kommen, ſonſt vergieng ſie in ihrer Angſt. == Die Säuaſie- wären. ſchon angekommen, und es fehlte ihr do < noch an alien, === Hartfopyf mußte ſich erſt TIM und endlich mach langem Rachdenken fiel es ihm erſt wieder ein, ==. Ex langte den Zettel aus dem- Accisbuche heraus, und ſagte ſeiner Frau, daß ſie das , was da wäre, herbeyſchaffen mschte. ZIudeß „mangelten doch einige Generalia, die alſo wegfielen. == ' Havrkopf gieng ſelbſt mit und überbrachte es == | „Herr Jemine! er böſer Mann 1-2 ſchrie ihm die Frau Amtmännin entgegen, und ſchlug ihn auf die Schulter und blickte den Korb, der - voll gedrückt, gerüttelt und überfirfſig in ihren Ss HEILE werden ſolite, mit Augen voller EE) Fron?"""

In [19]:
print('{:.4f}'.format(character_error_rate(gt, tess)))

0.1190


XXX This gives a smaller CER than ocrevalUAtion (which gives 0.1228). Why?

In [20]:
levenshtein(gt, tess)/len(gt)

0.1190253045923149

That's ~ the same, so I think it's not about the character segmentation. Check that we're only dealing with single-codepoint grapheme clusters:

In [21]:
for w in gt, tess:
    for g in grapheme_clusters(w):
        assert len(g) == 1

Maybe ocrevalUAtion doesn't count whitespace?

In [22]:
def remove_whitespace(s):
    return s.replace(' ', '')
remove_whitespace(gt)

'115überdievielenSorgenwegendeſſelbenvergaßHartkopf,derFrauAmtmännindasver⸗ſprochenezuüberliefern.—EinErpreſſerwurdeanihnabgeſchickt,umihnumsHim⸗melswillenzuſagen,daßerdasVerſprochenegleichdenAugenblicküberbringenmöchte,dieFrauAmtmänninhätteſichaufihnverlaſſen,undnunwüßteſienicht,wasſieanfangenſollte.DenAugenblickſollteerkommen,ſonſtvergiengſieinihrerAngſt.—DieGäſtewärenſchonangekommen,undesfehlteihrdochnochanallem.—Hartkopfmußteſicherſtbeſinnen,undendlichnachlangemNachdenkenfielesihmerſtwiederein.—ErlangtedenZettelausdemAccisbucheheraus,undſagteſeinerFrau,daßſiedas,wasdawäre,herbeyſchaffenmöchte.JndeßmangeltendocheinigeGeneralia,diealſowegfielen.—Hartkopfgiengſelbſtmitundüberbrachtees.—„HerrJemine!erböſerMann!“—ſchrieihmdieFrauAmtmänninentgegen,undſchlugihnaufdieSchulterundblicktedenKorb,dervollgedrückt,gerütteltundüberﬂüſſiginihrenSchoosgegebenwerdenſollte,mitAugenvollerFreu⸗H2'

In [23]:
print('{:.4f}'.format(character_error_rate(remove_whitespace(gt), remove_whitespace(tess))))

0.1324


Now it's larger than ocrevalUAtion 🤷‍♂️

# Word error rate

## Word segmentation

Naively split on spaces.

(Note: ocrevalUAtion does confusing things here, like the Token splitting in a hash function, with an empty pattern?!)

In [24]:
def naive_word_split(s):
    return s.split(' ')

In [25]:
example_text = "The quick (“brown”) fox can't jump 32.3 feet, right?"

In [26]:
naive_word_split(example_text)

['The',
 'quick',
 '(“brown”)',
 'fox',
 "can't",
 'jump',
 '32.3',
 'feet,',
 'right?']

Let's do it the Unicode way (Appendix UAX #29 on Unicode Text Segmentation): Split on word boundaries using the uniseg libraries and ignore words that contain only whitespace, punctuation "and similar characters":

In [27]:
from word_error_rate import words
print(inspect.getsource(words))

list(words(example_text))

def words(s):
    # Patch uniseg.wordbreak.word_break to deal with our private use characters. See also
    # https://www.unicode.org/Public/UCD/latest/ucd/auxiliary/WordBreakProperty.txt
    old_word_break = uniseg.wordbreak.word_break

    def new_word_break(c, index=0):
        if 0xE000 <= ord(c) <= 0xF8FF:  # Private Use Area
            return 'ALetter'
        else:
            return old_word_break(c, index)
    uniseg.wordbreak.word_break = new_word_break

    # Check if c is an unwanted character, i.e. whitespace, punctuation, or similar
    def unwanted(c):

        # See https://www.fileformat.info/info/unicode/category/index.htm
        # and https://unicodebook.readthedocs.io/unicode.html#categories
        unwanted_categories = 'O', 'M', 'P', 'Z', 'S'
        unwanted_subcategories = 'Cc', 'Cf'

        subcat = unicodedata.category(c)
        cat = subcat[0]
        return cat in unwanted_categories or subcat in unwanted_subcategories

    # We follow Unicode Standard A

['The', 'quick', 'brown', 'fox', "can't", 'jump', '32.3', 'feet', 'right']

In [28]:
list(words('Der schnelle [„braune“] Fuchs kann keine 3,14 Meter springen, oder?'))

['Der',
 'schnelle',
 'braune',
 'Fuchs',
 'kann',
 'keine',
 '3,14',
 'Meter',
 'springen',
 'oder']

In [29]:
list(words('Dies ist ein Beispielsatz. Oh, ja.'))

['Dies', 'ist', 'ein', 'Beispielsatz', 'Oh', 'ja']

It's probably not correct for Chinese and Japanese, but at least it doesn't rely on spaces.

In [30]:
list(words('我很高興跟你見面'))  # "Pleased to meet you" in Mandarin, Traditional writing

['我', '很', '高', '興', '跟', '你', '見', '面']

In [31]:
list(words('医者を呼んでください。'))

['医', '者', 'を', '呼', 'ん', 'で', 'く', 'だ', 'さ', 'い']

## Word error rate

For the word error rate, normalize again and compare sequences of words.

In [32]:
from word_error_rate import word_error_rate
print(inspect.getsource(word_error_rate))

def word_error_rate(reference, compared):
    if isinstance(reference, str):
        reference_seq = list(words_normalized(reference))
        compared_seq = list(words_normalized(compared))
    else:
        reference_seq = list(reference)
        compared_seq = list(compared)

    d = levenshtein(reference_seq, compared_seq)
    if d == 0:
        return 0

    n = len(reference_seq)
    if n == 0:
        return float('inf')

    return d / n



In [33]:
word_error_rate('Dies ist ein Beispielsatz.', 'Dies isi ein Beispielsatz,')

0.25

In [34]:
word_error_rate('Fnord ist verdampfter Kräutertee!', 'Fnòrd ist verdmpfter Krautertee.')

0.75

In [35]:
word_error_rate(gt, tess)

0.18823529411764706

This is a little larger than the ocrevalUAtion result!